# Videos and Exercises for Session 3: Data Structuring in Pandas II

In this combined teaching module and exercise set, you will continue working with structuring data. 

In the last session, you were working with making operations on relatively clean data. However, before it is meaningful to make such operations, you will (as a data scientist) often have to do some very preliminary cleaning, involving for instance dealing with missings and duplicates as well as combining and restructuring larger sets of data.

In this session, you will learn how to do such data processing. This notebook is structured as follows:
1. Missings and Duplicated Data:
    - Handling Missings: Delete or Interpolate?
    - Spotting and Interpreting Duplicates
2. Combining Data Sets:
    - Intro to `merge`, `concat` and `join`
    - Horizontal and Vertical Merging
3. Split-Apply-Combine
    - Finding Means and Other Characteristics from Data Subsets (aggregation)
4. Reshaping Data
    - Wide and Long Data
    - Intro to `stack`

# Packages
Before we get started.... load in the required modules and set up the plotting library!

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import YouTubeVideo

#  Part 1: Duplicates and Missings

In this section we will use [this dataset](https://archive.ics.uci.edu/ml/datasets/Adult) from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.html) to practice some basic operations on pandas dataframes. This is an extract from the US Census containing almost 50,000 rwos of individual-level micro data from 1994.


> **Ex. 3.1.1:** This link `'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'` leads to a comma-separated file with income data from a US census. Load the data into a pandas dataframe and show the 25th to 35th row.

> _Hint 1:_ There are no column names in the dataset. Use the list `['age','workclass', 'fnlwgt', 'educ', 'educ_num', 'marital_status', 'occupation','relationship', 'race', 'sex','capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'wage']` as names. 

> _Hint 2:_ When you read in the csv, you might find that pandas includes whitespace in all of the cells. To get around this include the argument `skipinitialspace = True` to `read_csv()`. 

In [ ]:
### BEGIN SOLUTION
url = f'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'

labels = ['age','workclass', 'fnlwgt', 'educ', 'educ_num', 'marital_status', 'occupation','relationship', 'race', 'sex','capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'wage']

df_census = pd.read_csv(url, header=None, skipinitialspace = True, names=labels)
df_census.iloc[24:35]
### END SOLUTION

##  Missing data

Often our data having information missing, e.g. one row lacks data on education for a specific person. Watch the video below about missing data type and get some simple tools to deal with the problem.

In [ ]:
YouTubeVideo('BLmXVUS8Ju0', width=640, height=360)

> **Ex. 3.1.2:** What is the missing value sign in this dataset? Replace all missing values with NA's understood by pandas. Then proceed to drop all rows containing any missing values with the `dropna` method. Store this new dataframe as `df_census_new`. How many rows are removed in this operation?

> _Hint 1:_ if this doesn't work as expected you might want to take a look at the hint for 4.2.1 again.
 
> _Hint 2:_ The NaN method from NumPy might be useful

In [ ]:
from numpy import NaN

### BEGIN SOLUTION
df_census_new = df_census.replace('?', NaN).dropna()
print(f"We have dropped {len(df_census) - len(df_census_new)} rows")
print(f"This amounts to {round((len(df_census) - len(df_census_new))/len(df_census)*100,2)} % of the rows")
### END SOLUTION

##  Duplicated Data

Watch the video below about duplicated data - in particular duplicated rows and how to handle them.

In [ ]:
YouTubeVideo('kjdd_TQSRME', width=640, height=360)

#  Video 2: Duplicated Data

> **Ex. 3.1.3:** Determine whether or not duplicated rows is a problem in the NOAA weather data and the US census data in the module. You should come up with arguments from the structure of the rows. 

In [ ]:
### BEGIN SOLUTION
print(f"The share of duplicate observations in the weather data is {round((len(load_weather(1863)) - len(load_weather(1863).drop_duplicates(['station','datetime'])))/len(load_weather(1863))*100,2)} %")
print(f"The share of duplicate observations in the US census data is {round((len(df_census_new) - len(df_census_new.drop_duplicates()))/len(df_census_new)*100,2)} %")
### END SOLUTION

# Part 2: Combining Data Sets

Below we hear about how different datasets can be combined into one, by merging on overlapping information that exists in both datasets. If you want to know more then you can look up Chapter 8, section 8.2 in Python for Data Analysis, 2ed.

**Note:** In the video, we are only dealing with one-to-one joins. This is a type of merge where there is only one row with a given merge key in each data frame. However, you may encounter situations where there are multiple rows that share the value of a merge key. In this situation, you may perform a one-to-many join or a many-to-many join that forms a Cartesian product of your rows. You can read more about these types of merges by visiting the Jake van der Plass [link](https://jakevdp.github.io/PythonDataScienceHandbook/03.07-merge-and-join.html) or looking into PDA, section 8.2.

In [ ]:
YouTubeVideo('ueP5K7zhUDU', width=640, height=360)

We continue with the final part of three exercises on structuring weather data. Use the function for fetching and structuring weather data that you used in part 2 of this exercise (i.e. exercise section 4.1).

> **Ex. 3.2.1:** Get the processed data from years 1870-1875 as a list of DataFrames. Generate a variable that denotes the year that the observations belong to. Convert the list into a single DataFrame by concatenating vertically. 

In [ ]:
### BEGIN SOLUTION
my_list = []
for year in range(1870,1876):
    df = load_weather(year)
    df['year'] = year
    my_list.append(df)

df_weather = pd.concat(my_list, axis=0)
### END SOLUTION

# Part 3: Split-Apply-Combine

Often we need to process information for a given individual, point in time etc. Instead of writing a loop over all the subsets of the data, we can use a more clever approach. Below we introduce the split-apply-combine framework and show how we can leverage it in pandas. If you want to know more then you can look up Chapter 10 in Python for Data Analysis, 2ed.

In [ ]:
YouTubeVideo('PVBzPwR5YT8', width=640, height=360)

*Note:* The Split-Apply-Combine method uses `.groupby()`. As indicated in the video, `.groupby()` is a method of pandas dataframes, meaning we can call it like so: `data.groupby('colname')`. The method groups your dataset by a specified column, and applies any following changes within each of these groups. For a more detailed explanation see [this link](https://www.tutorialspoint.com/python_pandas/python_pandas_groupby.htm). The [documentation](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) might also be useful.

> **Ex. 3.3.1:** Use the *split-apply-combine*-method to compute the mean maximum daily temperature for each month-year pair. 

In [ ]:
### BEGIN SOLUTION
split_var=['year', 'month']
apply_var='obs_value'
df_weather.groupby(split_var)[apply_var].mean()
### END SOLUTION

> **Ex. 3.3.2:** Plot the monthly max,min, mean, first and third quartiles for maximum temperature for our station with the ID _'ITE00100550'_ for the years 1870-1875. 

> *Hint*: the method `describe` computes all these measures.

In [ ]:
### BEGIN SOLUTION
split_var = ['year', 'month']
apply_var = 'obs_value'
types = ['max', 'min', 'mean', '25%', '75%']
df_weather[df_weather['station']=='ITE00100550'].groupby(split_var)[apply_var].describe()[types].plot(figsize=(11,6))
### END SOLUTION

> **Ex. 3.3.3:** Parse the station location data which you can find at https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt. Merge station locations onto the weather data spanning 1870-1875.  

> _Hint:_ The location data have the folllowing format, 

```
------------------------------
Variable   Columns   Type
------------------------------
ID            1-11   Character
LATITUDE     13-20   Real
LONGITUDE    22-30   Real
ELEVATION    32-37   Real
STATE        39-40   Character
NAME         42-71   Character
GSN FLAG     73-75   Character
HCN/CRN FLAG 77-79   Character
WMO ID       81-85   Character
------------------------------
```

> *Hint*: The station information has *fixed width format* - does there exist a pandas reader for that? Here Google might be helpful!

In [ ]:
### BEGIN SOLUTION
url = f'https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt'
labels = ['station', 'latitude', 'longitude', 'elevation', 'state', 'name', 'gsn flag', 'hcn/crn flag', 'wmo id']

df_location = pd.read_fwf(url, header=None, names = labels)

df_weather_new = pd.merge(df_weather, df_location, on='station', how='left')
### END SOLUTION

##  Bonus Exercises

If you want to practice *split-apply-combine* a bit more before proceeding, we have generated two additional exercises that use the US census data from Part 1. In order to solve this exercise, you should use `df_census_new` which you generated in the last exercise.

> **Ex. 3.3.4:** (_Bonus_) Is there any evidence of a gender-wage-gap in the data? Create a table showing the percentage of men and women earning more than 50K a year.

In [ ]:
### BEGIN SOLUTION
df_census_new['HighWage'] = (df_census_new['wage'] == '>50K').astype(int)
df_census_new[['sex', 'HighWage']].groupby('sex').mean()
### END SOLUTION

> **Ex. 3.3.5:** (_Bonus_) Group the data by years of education (`educ_num`) and marital status. Now plot the share of individuals who earn more than 50K for the two groups 'Divorced' and 'Married-civ-spouse' (normal marriage). Your final result should look like this: 

![](examplefig.png)

> _Hint:_ remember the `.query()` method is extremely useful for filtering data.

In [ ]:
### BEGIN SOLUTION
df_census_new[['marital_status', 'HighWage', 'educ_num']]\
        .groupby(['marital_status', 'educ_num'])\
        .mean()\
        .reset_index()\
        .query("marital_status == 'Divorced' | marital_status == 'Married-civ-spouse'")\
        .set_index('educ_num')\
        .groupby('marital_status')\
        .HighWage\
        .plot()

plt.xlabel('Years of education')
plt.ylabel('Share earning more than 50K')
plt.legend()
### END SOLUTION

# Part 4: Reshaping  Dataframes

Often we have data that comes in a format that does not fit our purpose. If you want to know more then you can look up Chapter 8, section 8.3 in Python for Data Analysis, 2ed.

In [ ]:
YouTubeVideo('6eANX7bA5Xs', width=640, height=360)

In this exercise, we continue working with the NOAA data that you worked with in the main section of part 3.

> **Ex. 4.4.1:** For which months was the temperature in general colder in 1870 vs. 1875?

> *Hint: you may use `unstack`.


In [ ]:
### BEGIN SOLUTION
split_var = ['year', 'month']
apply_var = 'obs_value'
types = ['mean']

yearly_data = df_weather.groupby(split_var)[apply_var].describe()[types].unstack()

for month in range(0,12):
    if yearly_data.iloc[0,month]>yearly_data.iloc[5,month]:
        print(f"In month {month+1}, it was warmer in 1870 than in 1875")
    else:
        print(f"In month {month+1}, it was warmer in 1875 than in 1870")
### END SOLUTION

# Part 5: Bonus Exercises $-$ Traffic Data in Copenhagen

In this second part set of exercises you will be working with traffic data from Copenhagen Municipality.

The municipality have made the data openly available through the [opendata.dk](http://www.opendata.dk/) platform. We will use the data from traffic counters to construct a dataset of hourly traffic. We will use this data to get basic insights on the development in traffic over time and relate it to weather. The gist here is to practice a very important skill in Data Science: being able to quickly fetch data from the web and structure it so that you can work with it. Scraping usually gets a bit more advanced than what we will do today, but the following exercises should give you a taste for how it works. The bulk of these exercise, however, revolve around using the Pandas library to structure and analyze data.

### Part 3.5.A: Getting some data to work with

Hence follows a simple scraping exercise where you (1) collect urls for datasets in the webpage listing data on traffic counters and (2) use these urls to load the data into one dataframe.

> **Ex. 3.5.1:** *(Bonus)* Using the requests module, extract the html markup of the webpage data.kk.dk/dataset/faste-trafiktaellinger and store it as a string in a new variable.

In [ ]:
### BEGIN SOLUTION
from requests import get 

url = 'https://www.opendata.dk/city-of-copenhagen/faste-trafiktaellinger'
resp = get(url)

html = resp.text
### END SOLUTION

> **Ex. 3.5.2:** *(Bonus)* Using the re module, extract a list of all the urls in the html string and store them in a new variable.

> _Hint:_ Try using the re.findall method. You may want to Google around to figure out how to do this. Searching for something along the lines of "extract all links in html regex python" and hitting the first StackOverflow link will probably get you farther than reading elaborate documentation.

In [ ]:
import re 

### BEGIN SOLUTION
rehits = re.findall(r'href=[\'"]?([^\'" >]+)', html)
### END SOLUTION

> **Ex. 3.5.3:** *(Bonus)* Create a new variable `datalinks` that only contains the links that point to downloadable traffic data sheets. Some links may be preset more than once on the page. To get the unique links use the `set()` function on `datalinks`.

> _Hint:_ You want to filter the results from above. For example to only include urls with the term 'download' in them.

In [ ]:
### BEGIN SOLUTION
datalinks = set([url for url in rehits if 'download' in url])
### END SOLUTION

> **Ex. 3.5.4:** *(Bonus)* Using pd.read_excel method, load the datasets into a list. Your resulting variable should hold a list of Pandas dataframes.

> _Hint:_ you may want to set the `skiprows` keyword argument. 

In [ ]:
### BEGIN SOLUTION
traffic_raw = [pd.read_excel(url, skiprows = 10) for url in datalinks]
### END SOLUTION

> **Ex. 3.5.5:** *(Bonus)* Merge the list of dataframes into a single dataframe.

In [ ]:
### BEGIN SOLUTION
final_data = pd.concat(traffic_raw)
final_data
### END SOLUTION

### Part 3.5.B Structuring your data

If you successfully completed the previous part, you should now have a dataframe with about 183.397 rows (if your number of rows is close but not the same, worry not—it matters little in the following). Well done! But the data is still in no shape for analysis, so we must clean it up a little.

161.236 rows (and 30 columns) is a lot of data. ~3.3 MB by my back-of-the-envelope calculations, so not "Big Data", but still enough to make your CPU heat up if you don't use it carefully. Pandas is built to handle fairly large dataframes and has advanced functionality to perform very fast operations even when the size of your data grows huge. So instead of working with basic Python we recommend working pandas built-in procedures as they are constructed to be fast on dataframes.

Nerd fact: the reason pandas is much faster than pure Python is that dataframes access a lower level programming languages (namely C, C++) which are multiple times faster than Python. The reason it is faster is that it has a higher level of explicitness and thus is more difficult to learn and navigate.

> **Ex. 3.5.6:** *(Bonus)* Reset the row indices of your dataframe so the first index is 0 and the last is whatever the number of rows your dataframe has. Also drop the column named 'index' and the one named `Spor`.

In [ ]:
### BEGIN SOLUTION
final_data = final_data\
        .reset_index()\
        .drop(['index', 'Spor'], axis = 1)
### END SOLUTION

> **Ex. 3.5.7:** *(Bonus)* Rename variables from Danish to English using the dictionary below.

```python 
dk_to_uk = {
    'Vejnavn':'road_name',
    '(UTM32)':'UTM32_north',
    '(UTM32).1':'UTM32_east',
    'Dato':'date',
    'Vej-Id':'road_id'
}
```

In [ ]:
### BEGIN SOLUTION
dk_to_uk = {
    'Vejnavn':'road_name',
    '(UTM32)':'UTM32_north',
    '(UTM32).1':'UTM32_east',
    'Dato':'date',
    'Vej-Id':'road_id'
}


final_data = final_data.rename(columns = dk_to_uk)
### END SOLUTION

Python is quite efficient. For example, when you create a new dataframe by manipulating an old one, Python notices that—apart from some minor changes—these two objects are almost the same. Since memory is a precious resource, Python will represent the values in the new dataframe as references to the variables in the old dataset. This is great for performance, but if you for whatever reason change some of the values in your old dataframe, values in the new one will also change—and we don't want that! Luckily, we can break this dependency.

> **Ex. 3.5.8:** *(Bonus)* Break the dependencies of the dataframe that resulted from Ex. 5.2.7 using the `.copy` method. Delete all other dataframes.

In [ ]:
### BEGIN SOLUTION
new_data = final_data.copy()

del final_data
### END SOLUTION

Once you have structured appropriately, something that you will want to do again and again is selecting subsets of the data. Specifically, it means that you select specific rows in the dataset based on some column values.

>**Ex. 3.5.9:** *(Bonus)* Create a new column in the dataframe called total that is True when the last letter of road_id is T and otherwise False.

> _Hint:_ you will need the `pd.Series.str` attribute for this.

In [ ]:
### BEGIN SOLUTION
new_data['total'] = (new_data.road_id.str[-1] == 'T')
### END SOLUTION

> **Ex. 3.5.10:** *(Bonus)* Select rows where total is True. Delete all the remaining observations.

In [ ]:
### BEGIN SOLUTION
total_data = new_data[new_data.total == True]

del new_data
### END SOLUTION

> **Ex. 3.5.11:** *(Bonus)* Make two datasets based on the lists of columns below. Call the dataset with spatial columns data_geo and the other data.

```python
# Columns for `geo_data`, stored in `geo_columns`
spatial_columns = ['road_name', 'UTM32_north', 'UTM32_east']

# Columns for `data`, stored in `select_columns`
hours = ['kl.{}-{}'.format(str(h).zfill(2), str(h+1).zfill(2)) for h in range(24)]
select_columns = ['road_name', 'date'] + hours
```

In [ ]:
### BEGIN SOLUTION
# Columns for `geo_data`, stored in `geo_columns`
spatial_columns = ['road_name', 'UTM32_north', 'UTM32_east']

# Columns for `data`, stored in `select_columns`
hours = [f'kl.{str(h).zfill(2)}-{str(h+1).zfill(2)}' for h in range(24)]
select_columns = ['road_name', 'date'] + hours

# selections
data_geo = total_data[spatial_columns]
data  = total_data[select_columns]

del total_data
### END SOLUTION

> **Ex. 3.5.12:** *(Bonus)* Drop the duplicate rows in data_geo.

In [ ]:
### BEGIN SOLUTION
data_geo.drop_duplicates(inplace = True)
### END SOLUTION

**Formatting: wide and narrow format**

When talking about two-dimensional data (matrices, tables or dataframes, we can call it many things), we can either say that it is in wide or long format (see explanation here, "wide" and "long" are used interchangably). In Pandas we can use the commands stack and unstack to move between these formats.

The wide format has the advantage that it often requires less storage and is easier to read when printed. On the other hand the long format can be easier for modelling, because each observation has its own row. Turns out that the latter is what we most often need.

> **Ex. 3.5.13:** *(Bonus)* Turn the dataset from wide to long so hourly data is now vertically stacked. Store this dataset in a dataframe called data. Name the column with hourly information hour_period. Your resulting dataframe should look something like this.

> _Hint:_ pandas' melt function may be of use.

In [ ]:
### BEGIN SOLUTION

# A bonus info: This function is written using optional type declaration. This
# is used to communicate what type the inputs should be. It doesn't affect the 
# functionality in any way, but it can sometimes increase readability. 

def gather(df: pd.DataFrame, key: str, value: str, cols: list) -> pd.DataFrame:
    """ This helper makes pythons melt() behave like R's gather()
    """
    id_vars = [ col for col in df.columns if col not in cols ]
    id_values = cols
    var_name = key
    value_name = value
    return pd.melt( df, id_vars, id_values, var_name, value_name )

data_long = gather(data, 'hour_period', 'value', hours)

# # using .stack() 
# # dropna = False in stack() is necessary for the dataframe to be exactly equal to the one above
# data_long = data.set_index(["road_name", "date"])\
#                 .stack(dropna=False)\
#                 .reset_index()\
#                 .rename(columns = {"level_2": "hour_period", 0: "value"})

### END SOLUTION

**Categorical data**

Categorical data can contain Python objects, usually strings. These are smart if you have variables with string observations that are long and often repeated, e.g. with road names.

> **Ex. 3.5.14:** *(Bonus)* Use the `.astype` method to convert the type of the road_name column to categorical.

In [ ]:
### BEGIN SOLUTION
data_long.road_name = data_long.road_name.astype('category')
### END SOLUTION

### Structure temporal data

Pandas has native support for working with temporal data. This is handy as much 'big data' often has time stamps which we can make Pandas aware of. Once we have encoded temporal data it can be used to extract information such as the hour, second etc.

> **Ex. 3.5.15:** *(Bonus)* Create a new column called hour which contains the hour-of-day for each row. 

In [ ]:
### BEGIN SOLUTION
data_long['hour'] = pd.to_datetime(data_long['hour_period'].str[3:5], format = '%H')
### END SOLUTION

>**Ex. 3.5.16:** *(Bonus)* Create a new column called time, that contains the time of the row in datetime format. Delete the old temporal columns (hour, hour_period, date) to save memory.

> _Hint:_ try making an intermediary series of strings that has all temporal information for the row; then use pandas to_datetime function where you can specify the format of the date string.

In [ ]:
### BEGIN SOLUTION
data_long['timestamp'] = pd.to_datetime(data_long.date + ' ' + data_long.hour\
                                                                        .dt\
                                                                        .hour\
                                                                        .astype(str)\
                                                                        .apply(lambda x: x.zfill(2)),
                                        format = '%d.%m.%Y %H')

data_long = data_long.drop(['hour', 'hour_period', 'date'], axis = 1)
### END SOLUTION

> **Ex. 3.5.17:** *(Bonus)* Using your time column make a new column called weekday which stores the weekday (in values between 0 and 6) of the corresponding datetime.

> _Hint:_ try using the dt method for the series called time; dt has some relevant methods itself.

In [ ]:
### BEGIN SOLUTION
data_long['weekday'] = data_long.timestamp.dt.weekday

### Statistical descriptions of traffic data

> **Ex. 3.5.18:** *(Bonus)* Print the "descriptive statistics" of the traffic column. Also show a kernel density estimate of the values.

> _Hint:_ Use the describe method of pandas dataframes for the first task. Use seaborn for the second. 

In [ ]:
import seaborn as sb

### BEGIN SOLUTION
print(data_long.value.describe())

sb.kdeplot(data_long.value.dropna())
plt.show()
### END SOLUTION

> **Ex. 3.5.19:** *(Bonus)* Which road has the most average traffic?

In [ ]:
### BEGIN SOLUTION
data_long.groupby('road_name')\
         .value.agg('mean')\
         .reset_index()\
         .sort_values('value', ascending = False)\
         .head(10)
### END SOLUTION

> **Ex. 3.5.20:** *(Bonus)* Compute annual, average road traffic during day hours (9-17). Which station had the least traffic in 2013? Which station has seen highest growth in traffic from 2013 to 2014?

In [ ]:
### BEGIN SOLUTION
# create year variable
data_long['year'] = data_long.timestamp.dt.year
data_long['hour'] = data_long.timestamp.dt.hour

# subset relevant years and times 
data_delta = data_long.query("year == 2013 | year == 2014")\
                      .query("hour >= 9 & hour <= 17")

# convert to wide format
data_delta = data_delta[['road_name', 'value', 'year']].groupby(['road_name', 'year'])\
                                                       .agg('mean')\
                                                       .reset_index()\

# create year variable
data_long['year'] = data_long.timestamp.dt.year
data_long['hour'] = data_long.timestamp.dt.hour

# subset relevant years and times 
data_delta = data_long.query("year == 2013 | year == 2014")\
                      .query("hour >= 9 & hour <= 17")

# convert to wide format
data_delta = data_delta[['road_name', 'value', 'year']].groupby(['road_name', 'year'])\
                                                       .agg('mean')\
                                                       .reset_index()\
                                                       .pivot('road_name', 'year', 'value')\
                                                       .reset_index()
# remove redundant column name after pivoting 
data_delta.columns.name = None

# calculate relative size
data_delta['growth'] = (data_delta[2014]/data_delta[2013] - 1)*100

# show
data_delta.sort_values('growth', ascending = False)
### END SOLUTION